<a href="https://colab.research.google.com/github/andreRibeiro1989/medium/blob/main/get_more_done_in_less_time/chatgpt_gradio_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 963.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.7/140.7 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 K

In [2]:
import requests

headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer sk-YOUR-KEY', # Your OpenAI's SK key goes here
}

In [3]:
def get_messages(text, task):
    if task=='Summarise':
        messages = [
            {'role':'user','content':f'"""{text}"""\n\nTl;dr'},    
        ]
    elif task=='Oneliner':
        messages = [
            {'role':'user','content':f'Write a single short sentence that explains the following text: """{text}"""'},    
        ]
    elif task=='Keywords':
        messages = [
            {'role':'user','content':f'Extract comma separated list of keywords from this text: """{text}"""'},    
        ]
    elif task=='Notes':
        messages = [
            {'role':'user','content':f'Extract detailed list of bullet points from this text: """{text}"""'},    
        ]
    elif task=='SEO':
        messages = [
            {'role':'user','content':(
                'Generate an engaging, SEO optimised, Title (<=60 characters), Subtitle (<=100 characters), and snippet (<=300 characters) in this format:'
                '\nTitle:<title>\nSubtitle:<subtitle>\nSnippet:<snippet>'
                f'\n\n"""{text}"""'
            )},    
        ]
    elif task=='Grammar':
        messages = [
            {'role':'user','content':(
                'Correct the following text for spelling and grammar.'
                'Perform as few changes as possible to the text.'
                'Add at the end the list of errors and fixes in this format:'
                '\nErrors:\n<error> => <fix>'
                f'\n\n"""{text}"""'
            )},
        ]
    elif task.startswith('Translate'):
        messages = [
            {'role':'user','content':f'Translate the following text to {task.split("-")[-1]}: """{text}"""'},    
        ]
    else:
        print(f"No idea what to do: {task}")
    return messages

In [4]:
import re

# gets a response from chatgpt api given an array of messages
def get_api_response(messages):
    json_data = {
        'model': 'gpt-3.5-turbo',
        'messages': messages, # array of {'role':role, 'content':message}
        'max_tokens': 2048, # generates up to ~1536 words
    }
    
    # call chat completions endpoint
    response = requests.post(
        'https://api.openai.com/v1/chat/completions', 
        headers=headers, 
        json=json_data,
    ).json()
    
    # get the response message from chatgpt
    output = response['choices'][0]['message']['content']
    
    # tidying up the output
    output = re.sub('(^[\n ]+)|([\n ]+$)', '', output)
    
    return output

In [5]:
import gradio as gr

# modifies the prompt to the specific task and request the response
# to the chatgpt turbo model
def update(text,task):
    messages = get_messages(text, task)
    output = get_api_response(messages)
    return output

# Gradio attempts to open a new port every time it runs, if the previous port
# is already in use. To ensure we keep using the same port we will attempt
# to close the app if it is open still.
try:
    app.close()
except Exception as e:
    pass

# defines a multi-task interface using Gradio
with gr.Blocks() as app:
    # Elements within a Row will be displayed horizontally
    # In here we will display two 'columns side by side.
    with gr.Row(): 
        # Elements within a Column will be displayed vertically
        # In here we will display a textbox, a dropdown and a button atop each other
        with gr.Column(): 
            # textbox needed to add our text to be processed
            inp = gr.Textbox(
                label='Input Text', 
                placeholder="Enter text",
                lines=6,
                max_lines=6,
            )
            # dropdown to select the task to perform
            task = gr.Dropdown(
                ["Summarise", "Oneliner", "Keywords", 'Notes', "SEO", "Grammar", "Translate-French"], 
                label="Task",
                value='Summarise'
            )
            # button to submit the job
            btn = gr.Button("Run")
        # Elements within a Column will be displayed vertically
        # In here we will only display a single textbox
        with gr.Column():
            # textbox needed to dispaly the processed (output) text
            out = gr.Textbox(
                label='Output Text',
                interactive=False, # 'False' is needed to disalow edits
                lines=15,
                max_lines=15,
            )
    # when the button defined above is clicked, the 'get_api_response' 
    # function is called. The function will receive the 'inp' text
    # and the 'task' as arguments. The output of this function will be
    # displayed in the 'out' textbox. 
    btn.click(fn=update, inputs=[inp,task], outputs=out)

# modifies css to remove the gradio footer
app.css="footer {visibility: hidden}"

# launches the app in a new local port
app.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>